<a href="https://colab.research.google.com/github/MadHatter999/ALGO_LiDAR3.0/blob/main/Copy_of_Feature_Extraction_Gray_Level_Co_occurrence_Matrix_(GLCM)_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1) Nous utiliserons la bibliothèque 'os' pour lire les noms de fichiers des dossiers et la bibliothèque 'cv2' (OpenCV) pour faire le traitement d'image**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
from PIL import Image , ImageDraw

In [ ]:
def NoiseGenerator(height, width,number):#generer une image bruit
    for i in range (number):
        array = np.random.randint(255, size=(height, width),dtype=np.uint8)
        image = Image.fromarray(array)
        #image.show()
        nom=str(i)+".png" 
        image.save("/noise"+nom)

In [ ]:
def decouper(image):#corp l'images en des morceaux 50*50
    width, height = image.size
    if height%50==0:
        i_max=height//50
    else:
        i_max=height//50+1

    if width%50==0:
        j_max=width//50
    else:
        j_max=width//50+1

#im1 = im.crop((left, top, right, bottom)) 
    for i in range(i_max):
        for j in range(j_max):
            if i==i_max:
                if j==j_max:
                    croped= image.crop((50*j+1, 50*i+1, width,height)) 
                else:
                    croped= image.crop((50*j+1, 50*i+1,50*(j+1)+1 ,height)) 
            else:
                if j==j_max:
                    croped= image.crop((50*j+1, 50*i+1, width,50*(i+1)+1)) 
                else:
                    croped= image.crop((50*j+1, 50*i+1,50*(j+1)+1 ,50*(i+1)+1)) 
            nom=chr(ord('a')+j)+"."+chr(ord('a')+i)+".png"
            croped.save("MyDrive/decoupage/"+nom)

In [ ]:
def ecrire(text):#retourner une image contenant la valeur d'un des indices
    img = Image.new('RGB', (48,48), color = (255,255,255))
    d = ImageDraw.Draw(img)
    d.text((1,20), text, fill=(0,0,0))

    im_size = img.size
    Add_Padd = 1
    new_im = Image.new('RGB', (im_size[0]+2*Add_Padd,im_size[1]+2*Add_Padd)) # black
    new_im.paste(img, (Add_Padd,Add_Padd))
    return(new_im)


In [ ]:
%cd ..
image=Image.open('home/dice.png')
%cd /gdrive
decouper(image)

/
/gdrive


In [ ]:
import numpy as np 
import cv2 
import os
import re

# -------------------- Utility function ------------------------
def normalize_label(str_):
    str_ = str_.replace(" ", "")
    str_ = str_.translate(str_.maketrans("","", "()"))
    str_ = str_.split("_")
    return ''.join(str_[:2])

def normalize_desc(folder, sub_folder):
    text = folder + " - " + sub_folder 
    text = re.sub(r'\d+', '', text)
    text = text.replace(".", "")
    text = text.strip()
    return text

def print_progress(val, val_len, folder, sub_folder, filename, bar_size=10):
    progr = "#"*round((val)*bar_size/val_len) + " "*round((val_len - (val))*bar_size/val_len)
    if val == 0:
        print("", end = "\n")
    else:
        print("[%s] folder : %s/%s/ ----> file : %s" % (progr, folder, sub_folder, filename), end="\r")
        

# -------------------- Load Dataset ------------------------
 
dataset_dir = '/gdrive'

imgs = [] #list image matrix 
labels = []
descs = []
for folder in os.listdir(dataset_dir):
    for sub_folder in os.listdir(os.path.join(dataset_dir, folder)):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, folder, sub_folder))
        len_sub_folder = len(sub_folder_files) - 1
        for i, filename in enumerate(sub_folder_files):
            img = cv2.imread(os.path.join(dataset_dir, folder, sub_folder, filename))
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            h, w = gray.shape
            ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
            crop = gray[ymin:ymax, xmin:xmax]
            
            resize = cv2.resize(crop, (0,0), fx=0.5, fy=0.5)
            
            imgs.append(resize)
            labels.append(normalize_label(os.path.splitext(filename)[0]))
            descs.append(normalize_desc(folder, sub_folder))
            
            print_progress(i, len_sub_folder, folder, sub_folder, filename)

**2) L'ensemble de données d'images de feuilles sera stocké dans la liste 'imgs' et les noms de feuilles seront stockés dans la liste 'labels'. Nous pouvons prévisualiser l'une des images de la manière suivante**

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(imgs[0])

**3) Ensuite, nous pouvons trouver la matrice GLCM et la métrique de texture globale de l'image de la feuille en utilisant les fonctions greycomatrix() et greycoprops() dans la bibliothèque Scikit-Image.**

In [ ]:
from skimage.feature import greycomatrix, greycoprops

# ----------------- calculate greycomatrix() & greycoprops() for angle 0, 45, 90, 135 ----------------------------------
def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature


# ----------------- call calc_glcm_all_agls() for all properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls = []
for img, label in zip(imgs, labels): 
    glcm_all_agls.append(
            calc_glcm_all_agls(img, 
                                label, 
                                props=properties)
                            )
 
columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)
        
columns.append("label")

**4) Ensuite, nous changeons le format de données glcm_features de liste en Dataframe à l'aide de la bibliothèque pandas**

In [ ]:
import pandas as pd 

# Create the pandas DataFrame for GLCM features data
glcm_df = pd.DataFrame(glcm_all_agls, 
                      columns = columns)
glcm_df['label'] = glcm_df['label']
sorted_df = glcm_df.sort_values(by=['label'], ascending=True)
df=sorted_df.reset_index(drop=True)
#df.columns
df["dissimilarity_0"][0]

2.166666666666667

In [ ]:

%cd ..
%cd "/home"

/
/home


In [ ]:
#!pwd


In [ ]:
width, height = image.size
if height%50==0:
    i_max=height//50
else:
    i_max=height//50+1

if width%50==0:
    j_max=width//50
else:
    j_max=width//50+1
for indexe in range(len(df.columns)):
    result = Image.new('RGB', (j_max*50,i_max*50))
    for j in range(j_max):
        for i in range(i_max):
            text=str(df[df.columns[indexe]][(i_max*j)+i])
            im=ecrire(text)
            result.paste(im,(j*50,i*50))
    nom=df.columns[indexe]+'.png'
    result.save(nom)




In [ ]:
df

,dissimilarity_0,dissimilarity_45,dissimilarity_90,dissimilarity_135,correlation_0,correlation_45,correlation_90,correlation_135,homogeneity_0,homogeneity_45,...,contrast_135,ASM_0,ASM_45,ASM_90,ASM_135,energy_0,energy_45,energy_90,energy_135,label
0,2.166667,3.1250,2.416667,1.3750,0.042945,-0.457021,0.046429,0.387826,0.307240,0.236693,...,2.7500,0.048611,0.042969,0.036458,0.058594,0.220479,0.207289,0.190941,0.242061,a.a
1,2.541667,4.2500,3.250000,1.7500,0.130176,-0.557692,-0.253968,0.136364,0.240851,0.087443,...,4.7500,0.035590,0.042969,0.026042,0.044922,0.188654,0.207289,0.161374,0.211948,a.b
2,1.750000,3.1250,3.041667,1.8125,0.404878,-0.167477,-0.291720,-0.019269,0.427338,0.182975,...,5.0625,0.035590,0.042969,0.032118,0.060547,0.188654,0.207289,0.179215,0.246063,a.c
3,2.583333,3.3750,2.458333,1.5625,0.012161,-0.450187,-0.060413,0.397113,0.300772,0.171221,...,4.8125,0.035590,0.056641,0.051215,0.072266,0.188654,0.237993,0.226308,0.268823,a.d
4,1.916667,3.6875,2.916667,1.6875,0.383199,-0.471077,-0.359773,0.105987,0.356028,0.161642,...,3.9375,0.037326,0.048828,0.031250,0.058594,0.193200,0.220971,0.176777,0.242061,a.e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.000000,0.0000,0.000000,0.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,o.f
146,0.000000,0.0000,0.000000,0.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,o.g
147,0.000000,0.0000,0.000000,0.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,o.h
148,0.000000,0.0000,0.000000,0.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,o.i
